## Web Scraping

*In this notebook I'll scraping the data that i will use for this project. The data is the **List of Walt Disney Pictures Films** from wikipedia, the website can be found at: [Walt Disney Pictures Films](https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films).*

In [1]:
# import libraries

from bs4 import BeautifulSoup as bs # library use to do the web scraping
import requests # library to make the requests

*The goal of this step is create a list of dictionaries with the title movie and some informations about the movie, this information can be found on the page in the right side below the image*  

In [2]:
# first it'll be made for one example, then it'll iterate for all movies

url = "https://en.wikipedia.org/wiki/Dumbo" # i'll be taking the Dumbo movie as example

r = requests.get(url)

dumbo_page = bs(r.content)
print(dumbo_page.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Dumbo - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"6ff764db-e358-4a32-a638-e9d19f1205dd","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Dumbo","wgTitle":"Dumbo","wgCurRevisionId":982557408,"wgRevisionId":982557408,"wgArticleId":172299,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: others","CS1 maint: BOT: original-url status unknown","Articles with short description","Short description matches Wikidata","Wikipedia indefinitely move-protected p

In [33]:
# after we load the page, we inspect the page and found the tags that matchs our objective.
info_movie = dumbo_page.find(class_ = "infobox vevent")
info_movie_ = info_movie.find_all("tr")

movie = {}
informations = []
for index, info in enumerate(info_movie_):
    
    if index == 0:
        movie['title'] = info.get_text(" ", strip = True)
        
    elif index == 1:
        continue # where is kept the image (we dont want it)
        
    else:
        content_key = info.find("th").get_text(" ", strip = True)
        content_value = info.find("td").get_text(" ", strip = True).replace("\xa0", " ")
        movie[content_key] = content_value
    
movie
        

{'title': 'Dumbo',
 'Directed by': 'Supervising director: Ben Sharpsteen Sequence directors: Norman Ferguson Wilfred Jackson Bill Roberts Jack Kinney Samuel Armstrong',
 'Produced by': 'Walt Disney',
 'Story by': 'Otto Englander Joe Grant Dick Huemer',
 'Based on': 'Dumbo, the Flying Elephant by Helen Aberson Harold Pearl',
 'Starring': 'Edward Brophy Herman Bing Margaret Wright Sterling Holloway Verna Felton Cliff Edwards James Baskett Nick Stewart Hall Johnson Jim Carmichael',
 'Narrated by': 'John McLeish',
 'Music by': 'Frank Churchill Oliver Wallace',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': 'October 23, 1941 ( 1941-10-23 ) (New York City) [1] October 31, 1941 ( 1941-10-31 ) (U.S.)',
 'Running time': '64 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$950,000 [2]',
 'Box office': '$1.3 million (est. United States/Canada rentals, 1941) [3]'}

<tr><th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size:110%;font-style:italic;">Dumbo</th></tr>